In [1]:
import numpy as np
import torch
from torch import nn, optim
import torch.optim as optim
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
import seaborn as sns
import matplotlib.pyplot as plt
import vtk
from vtk import *
from vtk.util.numpy_support import vtk_to_numpy
import random
import os
import sys
import time
import csv
from argparse import Namespace

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print('Device running:', device)

Device running: cuda


In [3]:
class SineLayer(nn.Module):
    def __init__(self, in_features, out_features, bias=True, is_first=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        # self.enable_dropout = enable_dropout
        # self.dropout_prob = dropout_prob
        self.in_features = in_features
        # if enable_dropout:
        #     if not self.is_first:
        #         self.dropout = nn.Dropout(dropout_prob)
        self.linear = nn.Linear(in_features, out_features, bias=bias)

        self.init_weights()

    def init_weights(self):
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features,
                                             1 / self.in_features)
            else:
                self.linear.weight.uniform_(-np.sqrt(6 / self.in_features) / self.omega_0,
                                             np.sqrt(6 / self.in_features) / self.omega_0)


    def forward(self, x):
        x = self.linear(x)
        # if self.enable_dropout:
        #     if not self.is_first:
        #         x = self.dropout(x)
        return torch.sin(self.omega_0 * x)

In [4]:
class ResidualSineLayer(nn.Module):
    def __init__(self, features, bias=True, ave_first=False, ave_second=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        # self.enable_dropout = enable_dropout
        # self.dropout_prob = dropout_prob
        self.features = features
        # if enable_dropout:
        #     self.dropout_1 = nn.Dropout(dropout_prob)
        self.linear_1 = nn.Linear(features, features, bias=bias)
        self.linear_2 = nn.Linear(features, features, bias=bias)
        self.weight_1 = .5 if ave_first else 1
        self.weight_2 = .5 if ave_second else 1

        self.init_weights()


    def init_weights(self):
        with torch.no_grad():
            self.linear_1.weight.uniform_(-np.sqrt(6 / self.features) / self.omega_0,
                                           np.sqrt(6 / self.features) / self.omega_0)
            self.linear_2.weight.uniform_(-np.sqrt(6 / self.features) / self.omega_0,
                                           np.sqrt(6 / self.features) / self.omega_0)

    def forward(self, input):
        linear_1 = self.linear_1(self.weight_1*input)
        # if self.enable_dropout:
        #     linear_1 = self.dropout_1(linear_1)
        sine_1 = torch.sin(self.omega_0 * linear_1)
        sine_2 = torch.sin(self.omega_0 * self.linear_2(sine_1))
        return self.weight_2*(input+sine_2)

In [5]:
class MyResidualSirenNet(nn.Module):
    def __init__(self, obj):
        super(MyResidualSirenNet, self).__init__()
        # self.enable_dropout = obj['enable_dropout']
        # self.dropout_prob = obj['dropout_prob']
        self.Omega_0=30
        self.n_layers = obj['n_layers']
        self.input_dim = obj['dim']
        self.output_dim = obj['total_vars']
        self.neurons_per_layer = obj['n_neurons']
        self.layers = [self.input_dim]
        for i in range(self.n_layers-1):
            self.layers.append(self.neurons_per_layer)
        self.layers.append(self.output_dim)
        self.net_layers = nn.ModuleList()
        for idx in np.arange(self.n_layers):
            layer_in = self.layers[idx]
            layer_out = self.layers[idx+1]
            ## if not the final layer
            if idx != self.n_layers-1:
                ## if first layer
                if idx==0:
                    self.net_layers.append(SineLayer(layer_in,layer_out,bias=True,is_first=idx==0))
                ## if an intermdeiate layer
                else:
                    self.net_layers.append(ResidualSineLayer(layer_in,bias=True,ave_first=idx>1,ave_second=idx==(self.n_layers-2)))
            ## if final layer   
            else:
                final_linear = nn.Linear(layer_in,layer_out)
                ## initialize weights for the final layer
                with torch.no_grad():
                    final_linear.weight.uniform_(-np.sqrt(6 / (layer_in)) / self.Omega_0, np.sqrt(6 / (layer_in)) / self.Omega_0)
                self.net_layers.append(final_linear)

    def forward(self,x):
        for net_layer in self.net_layers:
            x = net_layer(x)
        return x

In [6]:
def size_of_network(n_layers, n_neurons, d_in, d_out, is_residual = True):
    # Adding input layer
    layers = [d_in]
    # layers = [3]

    # Adding hidden layers
    layers.extend([n_neurons]*n_layers)
    # layers = [3, 5, 5, 5]

    # Adding output layer
    layers.append(d_out)
    # layers = [3, 5, 5, 5, 1]

    # Number of steps 
    n_layers = len(layers)-1
    # n_layers = 5 - 1 = 4

    n_params = 0

    # np.arange(4) = [0, 1, 2, 3]
    for ndx in np.arange(n_layers):

        # number of neurons in below layer
        layer_in = layers[ndx]

        # number of neurons in above layer
        layer_out = layers[ndx+1]

        # max number of neurons in both the layer
        og_layer_in = max(layer_in,layer_out)

        # if lower layer is the input layer 
        # or the upper layer is the output layer
        if ndx==0 or ndx==(n_layers-1):
            # Adding weight corresponding to every neuron for every input neuron
            # Adding bias for every neuron in the upper layer
            n_params += ((layer_in+1)*layer_out)
        
        else:

            # If the layer is residual then proceed as follows as there will be more weights if residual layer is included
            if is_residual:
                # doubt in the following two lines
                n_params += (layer_in*og_layer_in)+og_layer_in
                n_params += (og_layer_in*layer_out)+layer_out

            # if the layer is non residual then simply add number of weights and biases as follows
            else:
                n_params += ((layer_in+1)*layer_out)
            #
        #
    #

    return n_params

In [7]:
def compute_PSNR(arrgt,arr_recon):
    diff = arrgt - arr_recon
    sqd_max_diff = (np.max(arrgt)-np.min(arrgt))**2
    snr = 10*np.log10(sqd_max_diff/np.mean(diff**2))
    return snr

In [8]:
def srs(numOfPoints, valid_pts, percentage, isMaskPresent, mask_array):
    
    # getting total number of sampled points
    numberOfSampledPoints = int((valid_pts/100) * percentage)

    # storing corner indices in indices variable
    indices = set()
    
    # As long as we don't get the required amount of sample points keep finding the random numbers
    while(len(indices) < numberOfSampledPoints):
        rp = random.randint(0, numOfPoints-1)
        if isMaskPresent and mask_array[rp] == 0:
            continue
        indices.add(rp)

    # return indices
    return indices

In [9]:
def findMultiVariatePSNR(var_name, total_vars, actual, pred):
    # print('Printing PSNR')
    tot = 0
    psnr_list = []
    for j in range(total_vars):
        psnr = compute_PSNR(actual[:,j], pred[:,j])
        psnr_list.append(psnr)
        tot += psnr 
        print(var_name[j], ' PSNR:', psnr)
    avg_psnr = tot/total_vars
    print('\nAverage psnr : ', avg_psnr)
     #this function is calculating the psnr of final epoch (or whenever it is called) of each variable and then averaging it
     #Thus individual epochs psnr is not calculated
                                         
    return psnr_list, avg_psnr

In [10]:
def compute_rmse(actual, predicted):
    mse = np.mean((actual - predicted) ** 2)
    return np.sqrt(mse)

def denormalizeValue(total_vars, to, ref):
    to_arr = np.array(to)
    for i in range(total_vars):
        min_data = np.min(ref[:, i])
        max_data = np.max(ref[:, i])
        to_arr[:, i] = (((to[:, i] * 0.5) + 0.5) * (max_data - min_data)) + min_data
    return to_arr

In [11]:
def makeVTI(data, val, n_predictions, n_pts, total_vars, var_name, dim, isMaskPresent, mask_arr, vti_path, vti_name, normalizedVersion = False):
    nn_predictions = denormalizeValue(total_vars, n_predictions, val) if not normalizedVersion else n_predictions
    writer = vtkXMLImageDataWriter()
    writer.SetFileName(vti_path + vti_name)
    img = vtkImageData()
    img.CopyStructure(data)
    if not isMaskPresent:
        for i in range(total_vars):
            f = var_name[i]
            temp = nn_predictions[:, i]
            arr = vtkFloatArray()
            for j in range(n_pts):
                arr.InsertNextValue(temp[j])
            arr.SetName(f)
            img.GetPointData().AddArray(arr)
        # print(img)
        writer.SetInputData(img)
        writer.Write()
        print(f'Vti File written successfully at {vti_path}{vti_name}')
    else:
        for i in range(total_vars):
            f = var_name[i]
            temp = nn_predictions[:, i]
            idx = 0
            arr = vtkFloatArray()
            for j in range(n_pts):
                if(mask_arr[j] == 1):
                    arr.InsertNextValue(temp[idx])
                    idx += 1
                else:
                    arr.InsertNextValue(0.0)
            arr.SetName('p_' + f)
            data.GetPointData().AddArray(arr)
        # print(data)
        writer.SetInputData(data)
        writer.Write()
        print(f'Vti File written successfully at {vti_path}{vti_name}')

In [12]:
def getImageData(actual_img, val, n_pts, var_name, isMaskPresent, mask_arr):
    img = vtkImageData()
    img.CopyStructure(actual_img)
    # if isMaskPresent:
    #     img.DeepCopy(actual_img)
    # img.SetDimensions(dim)
    # img.SetOrigin(actual_img.GetOrigin())
    # img.SetSpacing(actual_img.GetSpacing())
    if not isMaskPresent:
        f = var_name
        data = val
        arr = vtkFloatArray()
        for j in range(n_pts):
            arr.InsertNextValue(data[j])
        arr.SetName(f)
        img.GetPointData().SetScalars(arr)
    else:
        f = var_name
        data = val
        idx = 0
        arr = vtkFloatArray()
        for j in range(n_pts):
            if(mask_arr[j] == 1):
                arr.InsertNextValue(data[idx])
                idx += 1
            else:
                arr.InsertNextValue(0.0)
        arr.SetName(f)
        img.GetPointData().SetScalars(arr)
    return img

In [13]:
# ----------------------------------------------------------------------------
# 1. Setup the parameter lists you want to iterate over
# ----------------------------------------------------------------------------
n_neurons_list = [480]
n_layers_list = [4, 8]      # These are the hidden layers (the code adds +2 internally)
batch_size_list = [512, 1024]
learning_rate_list = [5e-5, 1e-5, 5e-6]

# You can also configure other fixed parameters here if you wish
MAX_EPOCH = 100
decay = True 
decay_rate = 0.8
decay_at_equal_interval = True
decay_interval = 15

# Paths
datapath = '/kaggle/input/fit-gmm3/gmm3.vti'  # Update to your data path
outpath = './models/'
dataset_name = '3d_data'
vti_name = 'predicted_vti'
vti_path = './data/pred.vti'


In [14]:
print("Reading .vti file from:", datapath)
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName(datapath)
reader.Update()

data = reader.GetOutput()
pdata = data.GetPointData()

n_pts = data.GetNumberOfPoints()
dim = data.GetDimensions()
n_dim = len(dim)
total_arr = pdata.GetNumberOfArrays()

print("n_pts:", n_pts, "dim:", dim, "n_dim:", n_dim, "total_arr:", total_arr)

var_name = []
data_array = []

# --- Extract arrays from .vti ---
for i in range(total_arr):
    a_name = pdata.GetArrayName(i)
    cur_arr = pdata.GetArray(a_name)
    n_components = cur_arr.GetNumberOfComponents()
    
    if n_components == 1:
        var_name.append(a_name)
        data_array.append(vtk_to_numpy(cur_arr))
    else:
        # If array has multiple components (e.g. vector), split them
        component_names = [f"{a_name}_{c}" for c in ['x','y','z'][:n_components]]
        var_name.extend(component_names)
        for c in range(n_components):
            c_data = [cur_arr.GetComponent(j, c) for j in range(n_pts)]
            data_array.append(np.array(c_data))

total_vars = len(var_name)
univariate = (total_vars == 1)

# Prepare coordinate and value arrays
cord = np.zeros((n_pts, n_dim), dtype=np.float32)
val = np.zeros((n_pts, total_vars), dtype=np.float32)

for i in range(n_pts):
    pt = data.GetPoint(i)  # (x, y, z) or (x, y) etc. if 2D
    cord[i, :] = pt
    val[i, :] = [arr[i] for arr in data_array]

print("Total Variables:", total_vars)
print("Univariate:", univariate)
print("Coordinates Shape:", cord.shape)
print("Values Shape:", val.shape)

Reading .vti file from: /kaggle/input/fit-gmm3/gmm3.vti
n_pts: 262144 dim: (64, 64, 64) n_dim: 3 total_arr: 9
Total Variables: 9
Univariate: False
Coordinates Shape: (262144, 3)
Values Shape: (262144, 9)


In [15]:
# --- Normalize values to [-1, 1] ---
for i in range(total_vars):
    min_data = np.min(val[:, i])
    max_data = np.max(val[:, i])
    val[:, i] = 2.0 * ((val[:, i] - min_data) / (max_data - min_data) - 0.5)

# --- Normalize coordinates to [-1, 1] ---
for i in range(n_dim):
    # Range is [0, dim[i]-1], so we divide by (dim[i]-1)
    cord[:, i] = 2.0 * (cord[:, i] / (dim[i] - 1) - 0.5)

# Convert numpy arrays to PyTorch tensors
torch_coords_full = torch.from_numpy(cord)
torch_vals_full = torch.from_numpy(val)

full_dataset = TensorDataset(torch_coords_full, torch_vals_full)

In [16]:
!rm -rf /kaggle/working/*

In [17]:
results_file = "experiment_results.csv"
# Check if results file exists to know if we need a header
write_header = not os.path.isfile(results_file)

In [18]:
# Create (or append) to CSV
with open(results_file, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    if write_header:
        writer.writerow(["n_neurons", "n_layers_total", "batch_size", "learning_rate", "psnr", "rmse"])
    
    # Nested loops for each hyperparam
    for n_neurons in n_neurons_list:
        for hidden_layers in n_layers_list:
            for batch_size in batch_size_list:
                for lr in learning_rate_list:

                    # ----------------------------------------------------------------
                    # 4. Set up a fresh training run for this particular combination
                    # ----------------------------------------------------------------
                    
                    # Here we simulate argparse with the chosen hyperparams
                    # We'll store `hidden_layers` in args.n_layers.
                    # The original script then does `n_layers = args.n_layers + 2`.
                    # That means total layers = hidden_layers + 2 (input + output).
                    args = Namespace(
                        n_neurons = n_neurons,
                        n_layers = hidden_layers,       # note: the code adds +2 inside
                        epochs = MAX_EPOCH,
                        batchsize = batch_size,
                        lr = lr,
                        no_decay = (not decay),
                        decay_rate = decay_rate,
                        decay_at_interval = decay_at_equal_interval,
                        decay_interval = decay_interval,
                        
                        datapath = datapath,
                        outpath = outpath,
                        exp_path = './logs/',  # or wherever you want
                        modified_data_path = './data/',
                        dataset_name = dataset_name,
                        vti_name = vti_name,
                        vti_path = vti_path
                    )

                    print("\n====================================================")
                    print("Starting training with hyperparams:")
                    print("n_neurons =", n_neurons)
                    print("hidden_layers =", hidden_layers, "(+2 = total layers in the net)")
                    print("batch_size =", batch_size)
                    print("learning_rate =", lr)
                    print("====================================================\n")

                    # ----------------
                    # Setup Dataloader
                    # ----------------
                    train_dataloader = DataLoader(
                        full_dataset,
                        batch_size=batch_size,
                        pin_memory=True,
                        shuffle=True,
                        num_workers=4
                    )

                    # Because the original code references these variables inside:
                    # We'll replicate the snippet that extracts them from `args`.
                    decay = not args.no_decay
                    MAX_EPOCH = args.epochs
                    n_layers_total = args.n_layers + 2  # input + hidden + output
                    BATCH_SIZE = args.batchsize
                    LR = args.lr

                    # Build the model config dictionary
                    model_config = {
                        'total_vars': total_vars,
                        'dim': n_dim,
                        'n_neurons': args.n_neurons,
                        'n_layers': n_layers_total
                    }

                    # Create model/optimizer/loss fresh each time
                    model = MyResidualSirenNet(model_config).to(device)
                    optimizer = optim.Adam(model.parameters(), lr=LR, betas=(0.9, 0.999))
                    criterion = nn.MSELoss()

                    # Training loop
                    best_loss = 1e8
                    best_epoch = -1
                    train_loss_list = []

                    for epoch in range(MAX_EPOCH):
                        model.train()
                        temp_loss_list = []
                        start = time.time()

                        for X_train, y_train in train_dataloader:
                            X_train = X_train.type(torch.float32).to(device)
                            y_train = y_train.type(torch.float32).to(device)

                            if univariate:
                                y_train = y_train.squeeze()

                            optimizer.zero_grad()
                            predictions = model(X_train)
                            predictions = predictions.squeeze()
                            loss = criterion(predictions, y_train)
                            loss.backward()
                            optimizer.step()

                            temp_loss_list.append(loss.detach().cpu().numpy())

                        epoch_loss = np.mean(temp_loss_list)

                        # Learning rate decay logic
                        if decay:
                            if decay_at_equal_interval:
                                if epoch >= decay_interval and epoch % decay_interval == 0:
                                    for param_group in optimizer.param_groups:
                                        param_group['lr'] *= decay_rate
                            else:
                                if epoch > 0 and epoch_loss > train_loss_list[-1]:
                                    for param_group in optimizer.param_groups:
                                        param_group['lr'] *= decay_rate

                        train_loss_list.append(epoch_loss)
                        if epoch_loss < best_loss:
                            best_loss = epoch_loss
                            best_epoch = epoch + 1

                        end = time.time()
                        print(f"[Epoch {epoch+1}/{MAX_EPOCH}] Loss: {epoch_loss:.6f} | "
                              f"Time: {end-start:.2f}s | LR: {optimizer.param_groups[0]['lr']:.8f}")

                    # After training finishes, do final predictions
                    print("\nFinished training.")
                    print("Best epoch:", best_epoch, "with loss:", best_loss)

                    # Prediction phase
                    model.eval()
                    prediction_list = [[] for _ in range(total_vars)]

                    group_size = 5000
                    with torch.no_grad():
                        for i in range(0, torch_coords_full.shape[0], group_size):
                            coords = torch_coords_full[i:i+group_size].type(torch.float32).to(device)
                            vals = model(coords)
                            vals = vals.cpu()

                            for j in range(total_vars):
                                prediction_list[j].append(vals[:, j])

                    extracted_list = [[] for _ in range(total_vars)]
                    for i in range(len(prediction_list[0])):
                        for j in range(total_vars):
                            el = prediction_list[j][i].detach().numpy()
                            extracted_list[j].append(el)
                    for j in range(total_vars):
                        extracted_list[j] = np.concatenate(extracted_list[j], dtype='float32')

                    n_predictions = np.array(extracted_list).T  # (n_pts, total_vars)

                    # Compute PSNR and RMSE (assuming these return numeric values)
                    psnr_value = findMultiVariatePSNR(var_name, total_vars, torch_vals_full.numpy(), n_predictions)
                    rmse_value = compute_rmse(torch_vals_full.numpy(), n_predictions)

                    print("PSNR:", psnr_value)
                    print("RMSE:", rmse_value)

                    # ------------------------------------------------------------
                    # Append one row to experiment_results.csv for this experiment
                    # ------------------------------------------------------------
                    writer.writerow([
                        n_neurons,
                        n_layers_total,  # total layers after +2
                        batch_size,
                        lr,
                        psnr_value,
                        rmse_value
                    ])

                    print("\n*** Results appended to 'experiment_results.csv' ***\n")
                    # End of hyperparameter combination

# End of script
print("All experiments completed. Check 'experiment_results.csv' for results.")


Starting training with hyperparams:
n_neurons = 480
hidden_layers = 4 (+2 = total layers in the net)
batch_size = 512
learning_rate = 5e-05

[Epoch 1/100] Loss: 0.005885 | Time: 4.07s | LR: 0.00005000
[Epoch 2/100] Loss: 0.002798 | Time: 3.08s | LR: 0.00005000
[Epoch 3/100] Loss: 0.002720 | Time: 3.19s | LR: 0.00005000
[Epoch 4/100] Loss: 0.002666 | Time: 3.49s | LR: 0.00005000
[Epoch 5/100] Loss: 0.002629 | Time: 3.37s | LR: 0.00005000
[Epoch 6/100] Loss: 0.002587 | Time: 3.38s | LR: 0.00005000
[Epoch 7/100] Loss: 0.002566 | Time: 3.46s | LR: 0.00005000
[Epoch 8/100] Loss: 0.002536 | Time: 3.38s | LR: 0.00005000
[Epoch 9/100] Loss: 0.002531 | Time: 3.36s | LR: 0.00005000
[Epoch 10/100] Loss: 0.002507 | Time: 3.45s | LR: 0.00005000
[Epoch 11/100] Loss: 0.002484 | Time: 3.23s | LR: 0.00005000
[Epoch 12/100] Loss: 0.002482 | Time: 3.13s | LR: 0.00005000
[Epoch 13/100] Loss: 0.002477 | Time: 2.99s | LR: 0.00005000
[Epoch 14/100] Loss: 0.002477 | Time: 3.14s | LR: 0.00005000
[Epoch 15/100

In [19]:
# print(os.path.getsize('/kaggle/working/models/train_3d_data_100ep_4rb_512n_512bs_1e-05lr_Truedecay_0.8dr_decayingAtInterval15.pth') / (1024 ** 2), 'MB')

In [20]:
# # vti saving path
# vti_path = args.vti_path
# if not os.path.exists(vti_path):
#     os.makedirs(vti_path)
# # vti name
# vti_name = args.vti_name
# makeVTI(data, nn_val, n_predictions, n_pts, total_vars, var_name, dim, isMaskPresent, mask_arr, vti_path, vti_name)